In [10]:
import csv

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
# 単語ベクトル化をGridSearchCVで使うのためのクラス
class myVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, method='tfidf', min_df=0.0005, max_df=0.10):
        self.method = method
        self.min_df = min_df
        self.max_df = max_df

    def fit(self, x, y=None):
        if self.method == 'tfidf':
            self.vectorizer = TfidfVectorizer(min_df=self.min_df, max_df=self.max_df)
        else:
            self.vectorizer = CountVectorizer(min_df=self.min_df, max_df=self.max_df)
        self.vectorizer.fit(x)
        return self

    def transform(self, x, y=None):
        return self.vectorizer.transform(x)

In [3]:
# GridSearchCV用パラメータ
PARAMETERS = [
    {
        'vectorizer__method':['tfidf', 'count'], 
        'vectorizer__min_df': [0.0003, 0.0004], 
        'vectorizer__max_df': [0.07, 0.10], 
        'classifier__C': [1, 3],    #10も試したが遅いだけでSCORE低い
        'classifier__solver': ['newton-cg', 'liblinear']},
    ]

# 時間がかかるので下記を省略
#        'vectorizer__min_df': [0.0003, 0.0004, 0.0005, 0.0006], 
#        'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},

In [4]:
# ファイル読込
def read_csv_column(col):
    with open('./sentiment_stem.txt') as file:
        reader = csv.reader(file, delimiter='\t')
        header = next(reader)
        return [row[col] for row in reader]    

In [5]:
x_all = read_csv_column(1)
y_all = read_csv_column(0)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all)

In [6]:
def train(x_train, y_train, file):
    pipline = Pipeline([('vectorizer', myVectorizer()), ('classifier', LogisticRegression())])
    
    # clf は classificationの略
    clf = GridSearchCV(
            pipline, # 
            PARAMETERS,           # 最適化したいパラメータセット 
            cv = 5)               # 交差検定の回数
    
    clf.fit(x_train, y_train)
    pd.DataFrame.from_dict(clf.cv_results_).to_csv(file)
    return clf.best_estimator_

In [7]:
def validate(estimator, x_test, y_test):
    
    for i, (x, y) in enumerate(zip(x_test, y_test)):
        y_pred = estimator.predict_proba([x])
        if y == np.argmax(y_pred).astype( str ):
            if y == '1':
                result = 'TP:正解がPositiveで予測もPositive'
            else:
                result = 'TN:正解がNegativeで予測もNegative'
        else:
            if y == '1':
                result = 'FN:正解がPositiveで予測はNegative'
            else:
                result = 'FP:正解がNegativeで予測はPositive'
        print(result, y_pred, x)
        if i == 29:
            break

In [8]:
%%time
estimator = train(x_train, y_train, 'gs_result.csv')

CPU times: user 41.5 s, sys: 192 ms, total: 41.7 s
Wall time: 41.8 s


In [11]:
%%time
validate(estimator, x_test, y_test)

TN:正解がNegativeで予測もNegative [[0.7839262 0.2160738]] restrain freak show mercenari obviou cerebr dull pretenti engag isl defi easi categor
FN:正解がPositiveで予測はNegative [[0.6469949 0.3530051]] chronicl man quest presid man singl handedli turn plane full hard bitten cynic journalist essenti campaign end extend public depart
TN:正解がNegativeで予測もNegative [[0.87843253 0.12156747]] insuffer movi mean make think existenti suffer instead put sleep
TN:正解がNegativeで予測もNegative [[0.90800564 0.09199436]] minut condens episod tv seri pitfal expect
TP:正解がPositiveで予測もPositive [[0.12240474 0.87759526]] absorb unsettl psycholog drama
TP:正解がPositiveで予測もPositive [[0.42977787 0.57022213]] rodriguez chop smart aleck film school brat imagin big kid
FN:正解がPositiveで予測はNegative [[0.59805784 0.40194216]] gangster movi capac surpris
TP:正解がPositiveで予測もPositive [[0.29473058 0.70526942]] confront stanc todd solondz take aim polit correct suburban famili
TP:正解がPositiveで予測もPositive [[0.21660554 0.78339446]] except act quiet